### Purpose

The intent of this notebook is to serve as a rapid testing ground for new utilities. Any logic written here should migrate to the src/ directory as proper functions.

#### Import and Constants

In [ ]:
import numpy as np

# Importing from the local code structure
import os
import sys
module_path = os.path.abspath(os.path.join("..", "src"))
sys.path.append(module_path)
from exercise_log.constants import *
from exercise_log.dataloader import DataLoader
from exercise_log.trend import Trendsetter
from exercise_log.utils import convert_mins_to_hour_mins
from exercise_log.vis import plot_resting_heart_rate, plot_strength_over_time, plot_weight, plot_workout_frequency

#### Load Data and Compute Trends

In [ ]:
EXTRAPOLATE_DAYS = 100
N_DAYS_TO_AVG = 8
ROOT_DATA_DIR = "../data"
ROOT_IMG_DIR = "../img"

health_metrics = DataLoader.load_health_metrics(ROOT_DATA_DIR)
travel_days = DataLoader.load_travel_days(ROOT_DATA_DIR)
cardio_workouts = DataLoader.load_cardio_workouts(ROOT_DATA_DIR)
weight_training_workouts = DataLoader.load_weight_training_workouts(ROOT_DATA_DIR)
weight_training_sets = DataLoader.load_weight_training_sets(ROOT_DATA_DIR)
all_workouts = DataLoader.load_all_workouts(cardio_workouts, weight_training_workouts, travel_days)

# n-day average over a week gives a sense of if I'm keeping above a relatively low baseline of 150 minutes/week
n_day_avg_workout_duration = Trendsetter.compute_n_sample_avg(all_workouts, DURATION, N_DAYS_TO_AVG)

# Fit relevant trendlines
weight_trendline = Trendsetter.fit_linear(health_metrics, WEIGHT, EXTRAPOLATE_DAYS)
heart_rate_trendline = Trendsetter.fit_logarithmic(health_metrics, RESTING_HEART_RATE, EXTRAPOLATE_DAYS)

#### Build Visuals

In [ ]:
# TODO Still need to build at least these visuals:
# * Walking data (max distance, max elevation gain, max duration, pace graph)
# * Strength Metrics grouped by workout
#    - Ideally: drop-down menu to select between various workouts
#    - Need to fine-tune some, remove ones that don't make sense (e.g. that have < 3 days worked)
plot_workout_frequency(all_workouts, n_day_avg_workout_duration, N_DAYS_TO_AVG, export_dir=ROOT_IMG_DIR)
plot_resting_heart_rate(all_workouts, health_metrics, heart_rate_trendline, EXTRAPOLATE_DAYS, export_dir=ROOT_IMG_DIR)
plot_weight(all_workouts, health_metrics, weight_trendline, EXTRAPOLATE_DAYS, export_dir=ROOT_IMG_DIR)

for exercise in weight_training_sets[EXERCISE].unique():
    print(f"Plotting {exercise}... ", end="")
    plot_strength_over_time(all_workouts, weight_training_sets, exercise, export_dir=ROOT_IMG_DIR, show_plot=False)
    print(f"done.")

# TODO [Workout frequency graph]
# Change colour of rest days to red or orange and MAYBE also color-code cardio vs weights
# Another idea would be generating sub-graphs for each step of the hierarchy. E.g.
# All -> Cardio -> Walk
# All -> Weight -> Chest (per muscle group, not pairs, bc I don't want to marry the visuals to the current splits)

#### Misc. Metrics

In [ ]:
# Construct a full-picture set of functional fitness metrics
# Include: 
#   1. Major strength metrics: 1RM deadlift, squat, bench press, strict press
#   2. Cardio metrics:
#        Walking: 1km time, 5km time, 1000ft climb time, max 1-min walking pace
#        Running: 1 mile time, 5km time, half-marathon time, marathon time, maximum duration
#        Swimming: 400m freestyle time?, 1000m freestyle time?
#        Cardio Health: 2-week average resting heart rate, V02 Max
#   3. Calisthenic metrics: max consecutive push-ups, pull-ups, burpees; static hang duration, vertical leap?
#   4. Minor strength metrics:
#        Chest: N/A
#        Arms: 1RM overhead tricep extension, preacher curl
#        Legs: 1RM leg curl, leg extension, seated calf raise
#        Back: 1RM lawnmowers, lat pulldown
#        Shoulder: 1RM arnold press
#        Forearms: grip strength


walk_workouts = cardio_workouts[cardio_workouts[WORKOUT_TYPE].isin({WALK_TREADMILL, WALK_OUTDOOR})]
distance_walked = round(walk_workouts[DISTANCE].to_numpy().sum())
avg_distance = np.average(walk_workouts[DISTANCE].to_numpy())
farthest = walk_workouts[DISTANCE].to_numpy().max()
avg_duration = round(np.average(walk_workouts[DURATION].to_numpy()))
avg_pace = np.average(walk_workouts[PACE].to_numpy())
fastest_pace = walk_workouts[PACE].to_numpy().max()
avg_duration_str = convert_mins_to_hour_mins(avg_duration // 60)
print("Walking Metrics")
print("Average distance: {:.2f}km".format(avg_distance))
print(f"Farthest distance: {farthest}km")
print(f"Total distance: {distance_walked}km")
print(f"Average duration: {avg_duration_str}")
print("Average pace (m/s): {:.2f}".format(avg_pace))
print("Fastest pace (m/s): {:.2f}".format(fastest_pace))

bike_workouts = cardio_workouts[cardio_workouts[WORKOUT_TYPE] == BIKE_STATIONARY]
distance_biked = round(bike_workouts[DISTANCE].to_numpy().sum())
print()
print("Biking Metrics")
print(f"Total distance: {distance_biked}km")

distance_travelled = distance_walked + distance_biked
print()
print("Summary Metrics")
print(f"Total distance travelled: {distance_travelled}km")